In [ ]:
import pandas as pd
import re

Data source : Data.ai - Mobile Games Statistics scrapped by converting in PDF and transforming the PDF to excel.

In [ ]:
def add_publisher_into_df(df):
    # Load the company list data into a DataFrame
    df_companies = pd.read_excel('XLS/Companies_List.xlsx')

    # Extract the 'Company' column into a set
    companies = set(df_companies['Company'])
    # Create the new 'Company' column
    df['Company'] = ''

    # Iterate through the companies
    for string in companies:
        # Find the index positions where the string matches at the end of 'Unified App'
        match_indices = df['Unified App'].str.endswith(string, na=False)
        # Update 'Company' column with matching strings from 'Unified App'
        df.loc[match_indices, 'Company'] = string
        # Remove the string from the end of 'Unified App'
        df['Unified App'] = df['Unified App'].str.replace(f'{string}$', '', flags=re.IGNORECASE).str.strip()

    return df

In [ ]:
#Creating year and month columns

In [ ]:
#List of DataSources
date_id = ["2307", "2306", "2305", "2304", "2303", "2302", "2301", "2212", "2211", "2210", "2209", "2208"]

for date in date_id:
    # 1 - Importing all Data from excel sheets
    data = pd.read_excel(f'XLS/Data_{date}.xlsx')

    # 2 - Removing all fully NaN Rows and Columns
    # Find rows containing only null values & Drop the null rows
    try:
        null_rows = data.rows[data.isnull().all()]
        df = data.drop(columns=null_columns)
    except:
        pass
    # Find columns containing only null values & Drop the null columns
    try:
        null_columns = data.columns[data.isnull().all()]
        df = data.drop(columns=null_columns)
    except:
        pass

    # 3 - Remove unwanted character and reasign corresponding value types
    # Specify the columns to clean
    columns_to_clean = ['Downloads', 'Cumulative Downloads', 'Revenue', 'Cumulative Revenue',
                        'Paid Downloads', 'Organic Downloads', 'Active Users', 'Install Base', 'Total Time']
    # Remove unwanted characters and convert numbers to their numerical equivalent
    for column in columns_to_clean:
        try:
            df[column] = df[column].str.replace(r'[\$]', '', regex=True)  # Remove '$' and '.'
        except:
            pass
        try:
            df[column] = df[column].str.replace(r'(\d+)([kK])', r'\g<1>e3', regex=True)  # Convert 'k' to '*1e3'
        except:
            pass
        try:
            df[column] = df[column].str.replace(r'(\d+)([mM])', r'\g<1>e6', regex=True)  # Convert 'm' to '*1e6'
        except:
            pass
        try:
            df[column] = df[column].str.replace(r'(\d+)([bB])', r'\g<1>e9', regex=True)  # Convert 'b' to '*1e9'
        except:
            pass
        try:
            df[column] = pd.to_numeric(df[column])  # Convert the column to numeric data type
        except:
            pass

    # 4 - Retrieving the Game's Genre in the 'Classification' Column and creating a column 'Genre'
    # Remove leading line break from 'Classification' and extract words after it into 'Genre'
    try:
        df['Genre'] = df['Classification'].str.split('\n').str[-1].str.strip()
        df['Classification'] = df['Classification'].str.replace('\n.*', '', regex=True)
    except:
        pass

    # 5 - Retrieving the Publisher of the Games and adding to a new column 'Publisher'
    #extract_publisher(df)
    add_publisher_into_df(df)

    # Save the resulting DataFrame to a new Excel file
    df.to_excel(f'Data_{date}.xlsx', index=False)
    df


In [11]:
data_view = pd.read_excel('Data_GamesCompanies/Data_2209.xlsx')
data_view.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 27 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Unnamed: 0              100 non-null    int64  
 1   Unified App             100 non-null    object 
 2   Classification          100 non-null    object 
 3   Downloads               99 non-null     float64
 4   Cumulative Downloads    100 non-null    int64  
 5   Revenue                 100 non-null    int64  
 6   Cumulative Revenue      100 non-null    int64  
 7   RPD
As of Sep 30, 2022  100 non-null    object 
 8   Paid Downloads          84 non-null     float64
 9   Organic Downloads       96 non-null     float64
 10   Paid Downloads         100 non-null    float64
 11  Organic Downloads.1     100 non-null    float64
 12  Usage Penetration       100 non-null    object 
 13  Active Users            100 non-null    int64  
 14  Install Penetration     100 non-null    flo

In [ ]:
# Check for null values in the 'Unified App' column
null_values = df['Unified App'].isnull().sum()

# Print the number of null values
print("Number of null values in 'Unified App' column:", null_values)

In [ ]:
# Check for null values in the 'Unified App' column
null_values = df['Company'].isnull().sum()

# Print the number of null values
print("Number of null values in 'Company' column:", null_values)

### Used to find all publisher missing from previous filtering

In [ ]:
# List of DataSources
date_id = ["2307", "2306", "2305", "2304", "2303", "2302", "2301", "2212", "2211", "2210", "2209", "2208"]
missing_companies = set()  # Set to store missing companies

for date in date_id:
    global missing_companies

    # 1 - Importing all Cleaned Data from excel sheets
    df = pd.read_excel(f'clean_data_{date}.xlsx')

    # Load the company list data into a DataFrame
    df_companies = pd.read_excel('XLS/games_company.xlsx')

    # Extract the 'Company' column into a set
    companies = set(df_companies['Company'])

    for index, row in df.iterrows():
        unified_app = row['Unified App']
        company = row['Company']

        # Check if the 'Company' value is NaN (empty)
        if pd.isna(company):
            missing_companies.add(unified_app)

# Convert the set to a DataFrame
missing_companies_df = pd.DataFrame({'Missing Companies': list(missing_companies)})

# Save the DataFrame to an Excel file
missing_companies_df.to_excel('missing_companies.xlsx', index=False)

print("Missing companies :", missing_companies)

In [ ]:
# Read 'missing_companies.xlsx'
missing_companies_df = pd.read_excel('XLS/missing_companies.xlsx')
# Retrieve the 'Missing Companies' column and store it in 'updated_companies' set
updated_companies = set(missing_companies_df['Company'])

# Read 'games_companies.xlsx'
games_companies_df = pd.read_excel('XLS/games_company.xlsx')
# Retrieve the 'Company' column and update the 'updated_companies' set
updated_companies.update(set(games_companies_df['Company']))

# Create a DataFrame from the 'updated_companies' set
company_list_df = pd.DataFrame({'Company': list(updated_companies)})

# Save the DataFrame to 'Company_list.xlsx'
company_list_df.to_excel('Company_list.xlsx', index=False)

updated_companies